# markdown.obisidian.personal.notes
> Functions for managing notes in an Obsidian.md math vault

In [ ]:
#| default_exp markdown.obsidian.personal.notes

In [ ]:
#| export
from typing import Union

from trouver.markdown.markdown.file import (
    MarkdownFile
)
from trouver.markdown.obsidian.links import (
    links_from_text
)
from trouver.markdown.obsidian.vault import (
    VaultNote
)


In [ ]:
from fastcore.test import *
from trouver.helper import _test_directory

## Get notes linked in notes

In [ ]:
#| export
def notes_linked_in_note(
        list_note: VaultNote, # The `VaultNote` in which to find the links.
        as_dict: bool = True # If `True`, returns a dict. Returns a list otherwise.
        ) -> Union[dict[str, VaultNote], list[VaultNote]]: # If dict, the keys are the names of the vault notes and the values are the `VaultNote` objects. If list, then the entries are the `VaultNote` objects.
    """Returns a list or dict of VaultNotes of notes linked by 
    a specified note.
    """
    text = list_note.text()
    links = links_from_text(text)
    if as_dict:
        return {link.file_name: VaultNote(list_note.vault, name=link.file_name) for link in links}
    else:
        return [VaultNote(list_note.vault, name=link.file_name) for link in links]


def notes_linked_in_notes_linked_in_note(
    list_note: VaultNote, # The `VaultNote` with links to notes containing the links to obtain.
    as_dict: bool = True # If `True`, returns a dict. Returns a list otherwise. 
    ) -> Union[dict[str, VaultNote], list[VaultNote]]: # If dict, the keys are the names of the vault notes and the values are the `VaultNote` objects. If list, then the entries are the `VaultNote` objects.
    """Returns a list or dict of VaultNotes of notes 
    linked by notes which are linked by a specified note.
    """
    linked_in_list_note = notes_linked_in_note(list_note, as_dict=False)
    notes_for_each_note = [notes_linked_in_note(note, as_dict=True) 
                           for note in linked_in_list_note]
    all_notes = {}
    for notes_for_note in notes_for_each_note:
        for name, note in notes_for_note.items():
            all_notes[name] = note
    if as_dict:
        return all_notes
    else:
        return [note for _, note in all_notes.items()]     

In [ ]:
# notes_linked_in_note(VaultNote(VAULT, name='ml_data_note_type_categorization'))
test_vault = _test_directory() / 'test_vault_3'
index_note = VaultNote(test_vault, name='_index_1_chapter_reference_1')

notes_as_dict = notes_linked_in_note(index_note)
assert isinstance(notes_as_dict, dict)
test_eq(len(notes_as_dict), 12)
for name, note in notes_as_dict.items():
    assert note.name == name
print(notes_as_dict)

notes_as_list = notes_linked_in_note(index_note)
test_eq(len(notes_as_list), 12)
print(notes_as_list)
# print(notes)

{'note_11': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_12': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_13': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_14': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_15': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_16': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_17': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_18': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_19': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_110': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_111': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_112': <trouver.markdown.obsidian.vault.VaultNote object>}
{'note_11': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_12': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_13': <trouver.markdown.obsidian.vault.VaultNote object>, 'note_14': <trouver.markdown.obsidian.vault.VaultN

In [ ]:
test_vault = _test_directory() / 'test_vault_3'
index_note = VaultNote(test_vault, name='_index_reference_1')

# This gets all the notes linked in the notes linked in `index_reference_1`.
# In other words, this gets all the notes linked in 
# - `_index_1_chapter_reference_1`
# - `_index_2_chapter_reference_1`, and
# - `_index_3_chapter_reference_1`.
notes_as_dict = notes_linked_in_notes_linked_in_note(index_note)
assert isinstance(notes_as_dict, dict)
test_eq(len(notes_as_dict), 17)
for name, note in notes_as_dict.items():
    assert note.name == name
assert 'note_without_a_section' in notes_as_dict
assert "note_titles don't really need much structure" in notes_as_dict


notes_as_list = notes_linked_in_notes_linked_in_note(index_note)
test_eq(len(notes_as_list), 17)

## Tags in notes

In [ ]:
#| export
def note_has_tag(
        note: VaultNote,
        tag: str # Without the hashtag `'#'`.
        ) -> bool:
    """Returns `True` if the note has the specified tag in its
    yaml frontmatter meta.
    """
    mf = MarkdownFile.from_vault_note(note)
    return mf.has_tag(tag)

In [ ]:
test_vault = _test_directory() / 'test_vault_3'
note = VaultNote(test_vault, name='i_am_an_independent_note_who_does_not_need_a_folder_and_that_is_okay_too')

assert note_has_tag(note, "this_is_a_tag")
assert not note_has_tag(note, "this_is_also_a_tag") # Although `#this_is_also_a_tag` is in the note, it is note in the yaml frontmatter meta.
assert note_has_tag(note, "this_is_a_tag/this_is_a_subtag")
assert not note_has_tag(note, "this_tag_is_not_in_the_note")